# Reconhecimento facial e classificação de objetos

Classificação de faces e classificação de objetos.

Alguns recursos e códigos foram adaptados deste [repositório](https://github.com/udacity/CVND_Exercises/) do curso de Visão Computacional da Udacity.

> Atenção: este notebook foi desenhado para funcionar no **Google Collab**. Se pretende executar localmente prefira a versão local deste notebook, sem o sufixo ```-collab```.

## 1. Requerimentos

### 1.1 Bibliotecas

Todas as bibliotecas já estão instaladas no Google Colab.

* OpenCV>=3.4.3
* Pillow>= 7.0.0
* Pytorch>=1.4.0
* Numpy>=1.18.1
* Keras >= 2.3.1
* Tensorflow >= 2.2.0
* Gdown >= 3.6.4

### 1.2 Arquivos

Baixe o repositório do GitHub utilizando o comando abaixo. Em caso de atualização, utilize o comando para apagar o diretório antes.

In [ ]:
!rm -rf fiap-ml-visao-computacional/

In [ ]:
!git clone https://github.com/michelpf/fiap-ml-visao-computacional

Vamos agora posicionar o diretório do repositório para a aula respectiva. Nesse caso envie o comando de mudança de diretório.

In [ ]:
%cd fiap-ml-visao-computacional/aula-5-machine-learning-aplicado/

Importação das bibliotecas.

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

#Exibição na mesma tela do Jupyter
%matplotlib inline

import datetime

from os import listdir
from os.path import isfile, join, isdir, sep

from tqdm import tqdm

from sklearn.metrics import accuracy_score
from imutils import paths, resize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.models import Sequential, load_model, model_from_json
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import Model, layers
from tensorflow.keras import optimizers

from utils import *
from darknet import Darknet

plt.style.use('seaborn')
sns.set_style("whitegrid", {'axes.grid' : False})

## 2. Classificação de Faces

Vamos fazer um estudo de benchmarking entre os 3 classificadores: Eingenfaces, Fihserfaces e LBPH.
Para entender melhor os pontos positivos de cada um deles, foi utilizado o [dataset de faces da FEI](https://fei.edu.br/~cet/facedatabase.html) de imagnes originais, sem nenhuma modificação. Ao todo são 4 arquivos anexados que possuem 14 imagens de 200 pessoas.
Neste estudo vamos utilizar somente a parte 1, que possui 50 sujeitos.

In [ ]:
# Exibição das imagens
img1 = cv2.imread("faces-fei/1-01.jpg")
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.imread("faces-fei/2-02.jpg")
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img3 = cv2.imread("faces-fei/3-03.jpg")
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
img4 = cv2.imread("faces-fei/4-04.jpg")
img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
img5 = cv2.imread("faces-fei/5-05.jpg")
img5 = cv2.cvtColor(img5, cv2.COLOR_BGR2RGB)
img6 = cv2.imread("faces-fei/6-06.jpg")
img6 = cv2.cvtColor(img6, cv2.COLOR_BGR2RGB)
img7 = cv2.imread("faces-fei/7-07.jpg")
img7 = cv2.cvtColor(img7, cv2.COLOR_BGR2RGB)
img8 = cv2.imread("faces-fei/8-08.jpg")
img8 = cv2.cvtColor(img8, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(40,20))

plt.subplot(241)
plt.imshow(img1)
plt.subplot(242)
plt.imshow(img2)
plt.subplot(243)
plt.imshow(img3)
plt.subplot(244)
plt.imshow(img4)
plt.subplot(245)
plt.imshow(img5)
plt.subplot(246)
plt.imshow(img6)
plt.subplot(247)
plt.imshow(img7)
plt.subplot(248)
plt.imshow(img8)
plt.show()

### 2.1 Identificação e segmentação de região de interesse

As imagens que iremos utilizar estão sem nenhum tratamento. Nosso objetivo aqui é ter um recorte somente do rosto de cada uma das pessoas, removendo o fundo e detalhes do vestuário.

Iremos aplicar o classificador de cascatada de Haar e extrair somente as faces de cada imagem, padronizar o tamanho e converter para escala de cinza, que é como os classificadores utilizados trabalham com as imagens. As faces extraídas deverão ser armazenadas na pasta ```treino``` e ```teste```, separando 70% para o treinamento do modelo e 30% para a validação.

> Importante: o classificador em cascada de Haar poderá detectar eventualmente falsos positivos, neste caso será fácil eliminar estes casos limitando a área mínima de cada uma delas. Após ensaios cheguei ao valor mínimo de área de rosto de **35000** pixels quadrados.

In [ ]:
classificador_face = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')

def extrator_face_bgr(imagem):
    
    imagem_gray = cv2.cvtColor(imagem,cv2.COLOR_BGR2GRAY)
    faces = classificador_face.detectMultiScale(imagem_gray, 1.2, 5)
    
    if faces is ():
        return None
    
    roi_list = []
    roi_area_list = []
    
    for (x,y,w,h) in faces:
        roi = imagem[y:y+h, x:x+w]
        area = h*w
        roi_area_list.append(area)
        roi_list.append(roi)

    max_area = 0
    max_area_id = 0
    
    for idx, area in enumerate(roi_area_list):
        if area > max_area:
            max_area = area
            max_area_id = idx
    
    if max_area < 35000:
        return None

    
    return roi_list[max_area_id]

A padronização da imagem é necessária pois no treinamento todas elas precisam estar no mesmo tamanho. Iremos aplicar o tamanho ```200x200 pixels``` e também iremos converter a imagem para escala de cinza.

In [ ]:
def padronizar_imagem(imagem):
    imagem_gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    imagem_gray = cv2.resize(imagem_gray, (200, 200), interpolation=cv2.INTER_LANCZOS4)
    return imagem_gray

Um pequeno ensaio de como as imagens serão transformadas.

In [ ]:
imagem = cv2.imread("faces-fei/1-03.jpg")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
plt.imshow(imagem)

In [ ]:
imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2BGR)
roi = extrator_face_bgr(imagem)
roi = cv2.cvtColor(roi, cv2.COLOR_RGB2BGR)
plt.imshow(roi)

In [ ]:
roi = padronizar_imagem(roi)
plt.imshow(roi, cmap="gray")

### 2.2 Separando dados de treinamento e teste

Cada sujeito (ou face) possui 14 imagens, destas pode existir algumas poses que o classificador não é capaz de identificar, como de lado. Desta forma, vamos trabalhar com imagens de índice até 8 no treino e acima para teste. Na prática, teremos 7 imagens para treino e 3 imagens para teste, alcançando o valor recomendável de validação de 30% do total de treino.

In [ ]:
# Carregando exemplos de arquivos previamente coletados
faces_path = "faces-fei/"
faces_path_treino = "faces-fei/treino/"
faces_path_teste = "faces-fei/teste/"

lista_arquivos = [f for f in listdir(faces_path) if isfile(join(faces_path, f))]

contador = 0

for arquivo in tqdm(lista_arquivos):
    imagem = cv2.imread(faces_path + arquivo)
    face = extrator_face_bgr(imagem)
   
    if face is not None:
        subject_num = int(arquivo.split(".")[0].split("-")[1])
        
        if subject_num < 9:
            cv2.imwrite(faces_path_treino + arquivo, face)
        else:
            cv2.imwrite(faces_path_teste + arquivo, face)

### 2.2 Preparação para o treinamento dos modelos

Com as imagens separadas, iremos reunir e organizá-las. Em ```lista_imagens_treino``` vamos armazenar todas as imagnes de treino. Na mesma sequência, armazenaremos a identificação do sujeito em ```lista_sujeitos_treino```. Cada sujeito tem uma identificação numérica única, por exemplo o arquivo ```1-03.jpg``` representa a terceira imagem do sujeito ```1```.

In [ ]:
lista_arquivos_treino = [f for f in listdir(faces_path_treino) if isfile(join(faces_path_treino, f))]

lista_sujeitos_treino = []
lista_imagens_treino = []

for arquivo in tqdm(lista_arquivos_treino):
    imagem = cv2.imread(faces_path_treino + arquivo)
    imagem = padronizar_imagem(imagem)
    lista_imagens_treino.append(imagem)
    sujeito = int(arquivo.split("-")[0])
    lista_sujeitos_treino.append(sujeito)

Total de imagens envolvidas no treinamento.

In [ ]:
len(lista_sujeitos_treino), len(lista_imagens_treino)

O mesmo processo para as imagens de treino.

In [ ]:
lista_arquivos_teste = [f for f in listdir(faces_path_teste) if isfile(join(faces_path_teste, f))]

lista_sujeitos_teste = []
lista_imagens_teste = []

for arquivo in tqdm(lista_arquivos_teste):
    imagem = cv2.imread(faces_path_teste + arquivo)
    imagem = padronizar_imagem(imagem)
    lista_imagens_teste.append(imagem)
    sujeito = int(arquivo.split("-")[0])
    lista_sujeitos_teste.append(sujeito)

Total de imagens envolvidas no teste.

In [ ]:
len(lista_sujeitos_teste), len(lista_imagens_teste)

Realizando a conversão dos rótulos que repressentam a identificação de cada sujeito. O formato ```array``` é requisito dos classificadores para treinarem.

In [ ]:
lista_sujeitos_treino = np.asarray(lista_sujeitos_treino, dtype=np.int32)
lista_sujeitos_teste = np.asarray(lista_sujeitos_teste, dtype=np.int32)

### 2.3 Treinamento e validação do modelo Eingenfaces

Treinamento e testes para validar a precisão do modelo Eingenfaces.

In [ ]:
modelo_eingenfaces = cv2.face.EigenFaceRecognizer_create()
modelo_eingenfaces.train(lista_imagens_treino, lista_sujeitos_treino)

Preparação dos testes de validação, que é a execução do modelo contro as imagens reservadas para teste.

In [ ]:
y_pred_eingenfaces = []

for item in tqdm(lista_imagens_teste):
    y_pred_eingenfaces.append(modelo_eingenfaces.predict(item)[0])
    
acuracia_eingenfaces = accuracy_score(lista_sujeitos_teste, y_pred_eingenfaces)
acuracia_eingenfaces

### 2.4 Treinamento e validação do modelo Fisherfaces

Treinamento e testes para validar a precisão do modelo Fisherfaces.

In [ ]:
modelo_lda = cv2.face.FisherFaceRecognizer_create()
modelo_lda.train(lista_imagens_treino, lista_sujeitos_treino)

In [ ]:
y_pred_lda = []

for item in tqdm(lista_imagens_teste):
    y_pred_lda.append(modelo_lda.predict(item)[0])
    
acuracia_lda = accuracy_score(lista_sujeitos_teste, y_pred_lda)
acuracia_lda

### 2.5 Treinamento e validação do modelo LBPH

Treinamento e testes para validar a precisão do modelo LBPH.

In [ ]:
modelo_lbph = cv2.face.LBPHFaceRecognizer_create()
modelo_lbph.train(lista_imagens_treino, lista_sujeitos_treino)

In [ ]:
y_pred_lbph = []

for item in tqdm(lista_imagens_teste):
    y_pred_lbph.append(modelo_lbph.predict(item)[0])
    
acuracia_lbph = accuracy_score(lista_sujeitos_teste, y_pred_lbph)
acuracia_lbph

Daddo que as imagens não seguem um padrão de captura, ou seja, estão com poses diferentes uma das outras e levando em consideração que em cada sujeito havia uma pose com baixa luminosidade, pudemos verificar que o classificador Eingenfaces não conseguiu obter precisão de 45%. Enquanto a abordagem por LDA, do classificador Fisherfaces, obteve 63%. Por outro lado, o classificador LBPH foi o mais robusto dentre os 3, alcançado 89% de precisão.

O teste a seguir é para verificarmos que além da identificação do sujeito, temos o valor da distância de similaridade. Com ela podemos também deduzir um limite máximo para determinar se a identificação é indeterminada.

Nos classificadores Eigenfaces e Fisherfaces, valores de distância até 35 e 45 são considerados bons. Ao passo que no classificador LBPH os valores de similaridades utiilizados estão entre os valores entre 40 e 50.

In [ ]:
imagem = cv2.imread("faces-fei/teste/2-09.jpg")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
roi = extrator_face_bgr(imagem)
roi_padronizado = padronizar_imagem(roi)

predicao = modelo_lbph.predict(roi_padronizado)
predicao

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(imagem)
plt.title("Sujeito: 02, Predição: sujeito " + str(predicao[0]) + " distância " + str(predicao[1]))

## 3. Deep learning aplicado a OCR

Para criar um modelo de Deep Learning para reconhecimento de caracteres, vamos utilizar a base conhecida gerada de um sistema de captcha que foi utilizado nos sistemas do Tribunal Regional do Trabalho de São Paulo.

Neste exemplo, vamos utilizar somente os caracteres que aparecem na maior parte dos desafios de captcha, que na coleta foram de 33 letras e números.

A arquitetura e alguns componentes foram adaptados deste [artigo](https://towardsdatascience.com/image-classification-in-10-minutes-with-mnist-dataset-54c35b77a38d) de Orhan Gazi Yalçın.

### 3.1 Identificando as classes

Como um gerador de captchas nem sempre explora todo o alfabeto, vamos identificar exatamente quais as letras são utilizadas para listar todas as possíveis classes deste problema.

In [ ]:
pasta_imagens_treino = "captcha/imagens/"
lista_arquivos_classes = [f for f in listdir(pasta_imagens_treino) if isdir(join(pasta_imagens_treino, f))]

lista_classes = list(set(lista_arquivos_classes))
print(lista_classes)
print(len(lista_classes))

### 3.2 Enquadramento de imagem

Vamos deixar uma borda de segurança entre as letras para evitar classificações indevidas.

In [ ]:
def redimensionar_borda(imagem, comprimento, altura):
    
    # Obtendo as dimensões da imagem
    (h, w) = imagem.shape[:2]

    # Vamos deixar as imagens quadradas, logo se o comprimento for maior que a altura
    # O resize orginal do OpenCV sempre trabalha com altura e comprimento
    # A função resize do imutils, dado comprimento ou altura, ajusta o outro parâmetro 
    # para crescer mantendo o aspecto de razão
    if w > h:
        imagem = resize(imagem, width=comprimento)
    else:
        imagem = resize(imagem, height=altura)

    # Ajustando a borda
    padW = int((comprimento - imagem.shape[1]) / 2)
    padH = int((altura - imagem.shape[0]) / 2)

    imagem = cv2.copyMakeBorder(imagem, padH, padH, padW, padW, cv2.BORDER_CONSTANT, value=[255,255,255])
    imagem = cv2.resize(imagem, (comprimento, altura), interpolation=cv2.INTER_LANCZOS4)

    return imagem

Exemplo de imagem com tamanho despadronizado.

In [ ]:
imagem = cv2.imread("captcha/imagens/i/000001_3ibaz.png")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

plt.imshow(imagem)

Aplicando a padronização mas **sem** margem na borda.

In [ ]:
imagem_sem_borda = cv2.resize(imagem, (20, 20), interpolation=cv2.INTER_LANCZOS4)
plt.imshow(imagem_sem_borda)

Aplicando a padronização mas **com** margem na borda.

In [ ]:
imagem_padronizada = redimensionar_borda(imagem, 20, 20)
plt.imshow(imagem_padronizada)

### 3.3 Treinamento

Colecionando imagens para treinamento e realizando pequenos ajustes para posterior uso na biblioteca de deep-learning do Keras.

In [ ]:
classes = []
dados_imagem = []

pasta_imagens_treino = "captcha/imagens/"
lista_imagens_arquivos = paths.list_images(pasta_imagens_treino)

for imagem_caminho in tqdm(lista_imagens_arquivos):
    # Obtendo imagem e convertendo para escala de cinza
    imagem = cv2.imread(imagem_caminho)
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    imagem = redimensionar_borda(imagem, 20, 20)

    # Adicionando uma terceira dimensão (Canal Normalizado) conforme especificação do Keras
    imagem = np.expand_dims(imagem, axis=2)

    # Obtendo a caractere pelo nome do diretório
    classe = imagem_caminho.split(sep)[-2]

    dados_imagem.append(imagem)
    classes.append(classe)

Ao todo temos as seguintes quantidades de exemplos:

In [ ]:
len(classes), len(dados_imagem)

Vamos também simplificar a informação de escala de cinza. Utilizaremos a forma normalizada, dividindo todos os valores por 255. Desta forma um pixel 100% branco seria 1, e outro 100% preto seria 0.

In [ ]:
dados_imagem = np.array(dados_imagem, dtype="float") / 255
classes = np.array(classes)

Realizando a divisão de treinamento e validação.

In [ ]:
(x_train, x_test, y_train, y_test) = train_test_split(dados_imagem, classes, test_size=0.3, random_state=0)
len(x_train), len(x_test)

O Keras trabalha com uma forma diferente dos dados. Ao invés de utilizar as 3 dimensões, precisaremos de mais uma dimensão para incluir as imagens que farão parte dos treinamentos e testes, obtendo **Número de Imagens, Comprimento, Largura, Canal Normalizado**.

In [ ]:
print("Formato de dados da API Keras", x_train[0].shape)
print('Imagens de treino (x) 20 x 20:', x_train.shape)
print('Quantidade de imagens de treino', x_train.shape[0])
print('Quantidade de imagens de treino', x_test.shape[0])

Agora vamos definir a entrada dos dados, neste caso precisa ser exatamente da mesma forma que as imagens forem treinadas. Isso é portante pois a rede neural estará preparada para inferir somente imagens com este tamanho.

In [ ]:
shape_entrada = (20, 20, 1)

Incluindo codificação *one-hot*, ou seja, um conjunto de dados que está associado as classes, logo um dos 33 caracteres será codificado com bit 1 de acordo com sua posição na lista.

In [ ]:
lb = LabelBinarizer().fit(y_train)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

Exemplo de uma amostra:

In [ ]:
print(lista_classes)
print([lista_classes[7]])

In [ ]:
y_train[1]

Abaixo vamos constuir um modelo simples, do zero. Como as imagens são bem simples, diversas arquiteturas funcionam. Quando lidamos com objetos mais complexos, é bem comum optarmos por arquiteturas abertas como por exemplo:

* [VGG](http://www.robots.ox.ac.uk/~vgg/research/very_deep/) de Oxford
* [ResNet](https://arxiv.org/abs/1512.03385) da Microsoft
* [Inception](https://github.com/google/inception) do Google
* [Xception](https://arxiv.org/abs/1610.02357) do Google

Depois de avaliar estas arquiteturas, é possível adapta-las para classificar imagens específicas, isso se dá alterando as últimas camadas. É o que chamamos também de **Transfer Learning**.

Neste [link](https://medium.com/@sidereal/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5) você pode encontrar mais sobre outras arquiteturas.

In [ ]:
numero_classes = len(lista_classes)
numero_classes

### 3.4 Arquitetura

Partimos de um modelo simples que na maioria das vezes resolve problemas de OCR como esse. Como foi citado, identificações mais complexas utilizamos outras aboragens ou evolução de uma arquitetura inicial como esta.

In [ ]:
# Construindo um modelo sequencial
model = Sequential()

# Este componente, se trata de um filtro ou uma camada convulacional. Ela será responsável por 
# colocar uma janela de kernel (5x5), navegar pela imagem e extrair a soma dos pixels de cada janela
# o passo para mover a janela, chamado Stride, por padrão é de um pixel
model.add(Conv2D(20, kernel_size=(5,5), padding="same", input_shape=shape_entrada, activation="relu"))

# A camada de Pooling (ou MaxPooling2D) tem o papel de reduzir a dimensionalidade. Neste caso, a partir 
# da etapa anterior, será dividia em grupos de 2 x 2 pixels e será obtida o maior valor deles
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

# Esta etapa conhecida como "achatamento" é onde abrimos os dados organizados em tabelas (ou matrizes) 
# para uma única linha
model.add(Flatten())

# A camada densa (ou Dense) conectará cada elemento da camada anterior e passará para a próxima
# camada com as classes existentes
model.add(Dense(128, activation="relu"))

# O Dropout é um ruído gerado para evitar overfitting
model.add(Dropout(0.2))

# A camada final, determinará qual classe escolher. Por tal razão ela possui a ativação Softmax, que retorna 
# a probabilidade por classe
model.add(Dense(numero_classes, activation="softmax"))
model.summary()

Para revisão de funções de ativação, em particular [Relu](https://matheusfacure.github.io/2017/07/12/activ-func/).

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=10, batch_size=5)

Os gráficos a seguir mostram convergência de acurácia para os dados de treinamento e validação.
Note que o valor do erro, diferentemente da acurácia, não é expressada em porcentagem, portanto erro < 1 é um ótimo valor.

In [ ]:
# Para deixar no formato do Seaborn os gráficos do Pyplot
sns.set()

# Exibindo dados de Acurácia/Precisão
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Exibindo dados de Perda
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Salvando o modelo para uso posterior. Mesmo imagens pequenas como essas levam vários minutos para treinar.

In [ ]:
# Salvando o modelo no formato HDf5
model.save("modelos/model_captcha.h5")

# Arquitetura das camadas em JSSON e pesos treinados em HDF5
model.save_weights("pesos/weights_captcha.h5")

Uma vez salvo o modelo, nesta etapa é só carregar.

In [ ]:
# carregando o modelo no formato HDf5
model = load_model("modelos/model_captcha.h5")  
model.load_weights("pesos/weights_captcha.h5")
model.summary()

### 3.5 Testes de validação
Vamos inferir algumas imagens para verificar visualmente como o classificador está se comportando.
Para isso definimos uma função para normalizar uma imagem do captcha, para extrair os ruídos e posteriomente cada uma das suas letras.

In [ ]:
def imagem_normalizada(caminho_imagem):

    imagem = cv2.imread(caminho_imagem)
    imagem_gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    
    imagem_suavizada = cv2.GaussianBlur(imagem_gray, (5, 5), 0)
    _, imagem_limiarizada =  cv2.threshold(imagem_suavizada, 148, 255, cv2.THRESH_BINARY_INV)
    _, imagem_limiarizada =  cv2.threshold(imagem_limiarizada, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    kernel = np.ones((2, 2), np.uint8)
    imagem_erodida = cv2.erode(imagem_limiarizada, kernel, iterations = 2)

    return imagem_erodida

### 3.6 Conversão para escala de cinza

In [ ]:
plt.axis('off')

imagem_caminho = "captcha/anotados/d85iq.png"

imagem_original = cv2.imread(imagem_caminho, cv2.IMREAD_GRAYSCALE)
plt.imshow(imagem_original, cmap='gray')

### 3.7 Suavização para preparação de limiarização

Esta operação visa remover os ruídos da imagem, como as linhas transversais e pequenos pontos.

In [ ]:
imagem = cv2.imread(imagem_caminho)
imagem_gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

imagem_suavizada = cv2.GaussianBlur(imagem_gray, (5, 5), 0)

plt.axis('off')
plt.imshow(imagem_suavizada, cmap='gray')

### 3.8 Limiarização

A limirização remove todos os ruídos baseado num valor de limiar.

In [ ]:
_, imagem_limiarizada =  cv2.threshold(imagem_suavizada, 148, 255, cv2.THRESH_BINARY_INV)

plt.axis('off')
plt.imshow(imagem_limiarizada, cmap='gray')

### 3.9 Inversão da imagem

Para se adequar as imagens de treinamento e para que fique mais nítido a visualização das letras.

In [ ]:
 _, imagem_limiarizada =  cv2.threshold(imagem_limiarizada, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

plt.axis('off')
plt.imshow(imagem_limiarizada, cmap='gray')

### 3.10 Erosão

Como a imagem está invertida, aplicamos uma erosão para intensificar as linhas e melhorar a nitidez.

In [ ]:
kernel = np.ones((2, 2), np.uint8)
imagem_erodida = cv2.erode(imagem_limiarizada, kernel, iterations = 2)

plt.axis('off')
plt.imshow(imagem_erodida, cmap='gray')

Execução da função. Neste caso não fizemos a inversão da imagem pois as letras foram treinadas com o fundo branco.

In [ ]:
imagem_norm = imagem_normalizada(imagem_caminho)

plt.axis('off')
plt.imshow(imagem_norm, cmap='gray')

### 3.11 Identificação de contornos

Com a imagem com as letras bem definidas, iremos aplicar o método Canny para extrair os contornos.

In [ ]:
imagem_borda = cv2.Canny(imagem_norm, 30, 200)
contornos, _ = cv2.findContours(imagem_borda, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

plt.axis('off')
plt.imshow(imagem_borda, cmap="gray")

In [ ]:
imagem_borda_contornos = imagem_borda.copy()
imagem_borda_contornos = cv2.cvtColor(imagem_borda_contornos, cv2.COLOR_GRAY2RGB)

cv2.drawContours(imagem_borda_contornos, contornos, -1, (0,255,0), 1)

plt.axis('off')
plt.imshow(imagem_borda_contornos)

In [ ]:
len(contornos)

### 3.12 Extração das letras

A função a seguir, analisará os contornos identificados e fará um filtro baseado no tamanho do contorno. Em algumas ocasiões é possível ter contornos identificados em pequenos ruídos que ainda passam pelo processo, mas como eles são pequenos são facilmente identificados e removidos.

In [ ]:
def obter_letras(imagem):
    
    contornos_letras = []
    
    imagem_borda = cv2.Canny(imagem, 30, 200)
    contornos, _ = cv2.findContours(imagem_borda, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contorno in contornos:
        (x, y, w, h) = cv2.boundingRect(contorno)

        area = int(w) * int(h)
        
        if area <250:
            continue
        
        contornos_letras.append((x, y, w, h))
    
    print("Identificado " + str(len(contornos_letras)) + " contornos válidos.")
    
    # Se detectar mais do que 5 letras, detecção inválida
    if len(contornos_letras) < 5 :
        return False
    
    contornos_letras = sorted(contornos_letras, key=lambda x: x[0])
    
    lista_imagem_letras = []
    
    for retangulo_letra in contornos_letras:
        x, y, w, h = retangulo_letra
        imagem_letra = imagem[y - 10:y + h + 30, x - 1:x + w + 1]
        lista_imagem_letras.append(imagem_letra)
        
    return lista_imagem_letras

O retorno da função é a lista de regiões de interesse das letras.

In [ ]:
imagem_letras = obter_letras(imagem_norm)

### 3.13 Validação com imagem completa

A função a seguir, dada uma imagem, vai padronizá-la e inferir letra a letra.

In [ ]:
def obter_predicao(imagem):
    
    imagem_norm = redimensionar_borda(imagem, 20, 20)
    prediction = model.predict(imagem_norm.reshape(1, 20, 20, 1))
    label = lb.inverse_transform(prediction)[0]

    return label

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(151)
plt.title(obter_predicao(imagem_letras[0]), fontdict={'fontsize': 20})
plt.imshow(imagem_letras[0], cmap="gray")
plt.axis('off')

plt.subplot(152)
plt.title(obter_predicao(imagem_letras[1]), fontdict={'fontsize': 20})
plt.imshow(imagem_letras[1], cmap="gray")
plt.axis('off')

plt.subplot(153)
plt.title(obter_predicao(imagem_letras[2]), fontdict={'fontsize': 20})
plt.imshow(imagem_letras[2], cmap="gray")
plt.axis('off')

plt.subplot(154)
plt.title(obter_predicao(imagem_letras[3]), fontdict={'fontsize': 20})
plt.imshow(imagem_letras[3], cmap="gray")
plt.axis('off')

plt.subplot(155)
plt.title(obter_predicao(imagem_letras[4]), fontdict={'fontsize': 20})
plt.imshow(imagem_letras[4], cmap="gray")
plt.axis('off')
plt.show()

## 4. Transfer learning para reconhecimento de imagens

Técnica de transfer learning aplicado a deep learnig para classificação de imagens, utilizando classificados com pesos já treinados disponibilizados no Keras.

Foram utilizadas imagens com tamanho 100 x 100 pixels, 1409 imagens por classe para o treinamento e 472 imagens por classe para validação.

### 4.1 Geradores de imagens

Os geradores utilizados foram aplicados para converter o tamanho adequado do modelo utilizado (224 x 224 pixels) como também para criar novos exemplos a partir das imagens no que chamamos de _data augmentation_, por meio de perturbações da imagem baseado em recorte (```shear```), zoom e orientação horizontal (```horizontal_flip```).

Conjunto de dados utilizado foi [este](https://www.kaggle.com/moltean/fruits), disponível no Kaggle.

*Adaptado deste [artigo](https://medium.freecodecamp.org/keras-vs-pytorch-avp-transfer-learning-c8b852c31f02), de Patryk Miziula*

Com a técnica de _data augmentation_ foram geradas 1212 imagens por classe. Ao todo, o número de imagens subiu de 794 para 2424.

In [ ]:
train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    "imagens-frutas/train",
    batch_size=32,
    class_mode="binary",
    color_mode='rgb',
    target_size=(224,224))

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
    "imagens-frutas/validation",
    shuffle=False,
    class_mode="binary",
    color_mode='rgb',
    target_size=(224,224))

Exibindo as classes identificadas.

In [ ]:
train_generator.class_indices

### 4.2. Construindo a rede neural baseado em modelo pré-treinado

O Keras já possui classes especializadas para os seguintes modelos de deep-learning treinados com o conjunto de dados [ImageNet](http://www.image-net.org/):
  
* Xception
* VGG16
* VGG19
* ResNet50
* InceptionV3
* InceptionResNetV2
* MobileNet
* DenseNet
* NASNet
* MobileNetV2

Mais detalhes, veja na [documentação do Keras](https://keras.io/applications/).

_O Keras se encarrega de baixar o modelo automaticamente, não é preciso baixar separadamente._

Note que o parâmetro ```include_top=False``` configura o modelo para não utilizar a camada densa original, pois será substituída pelas novas classes.

In [ ]:
conv_base = VGG16(include_top=False)

for layer in conv_base.layers:
    layer.trainable = False

Removendo a camada densa para que seja adaptada para lidar com apenas 2 classes.

In [ ]:
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(3, activation='softmax')(x)
model = Model(conv_base.input, predictions)

model.summary()

In [ ]:
optimizer = optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### 4.3 Treinamento do modelo

In [ ]:
history = model.fit(train_generator, epochs=5, validation_steps=3, steps_per_epoch=3, validation_data=validation_generator)

Armazenamento do modelo e carregamento do modelo pré-treinado.

In [ ]:
# salvando o modelo no formato HDf5
model.save('modelos/model-frutas.h5')

# arquitetura das camadas em JSSON e pesos treinados em HDF5
model.save_weights('modelos/weights-frutas.h5')
with open('modelos/architecture-frutas.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
# carregando o modelo no formato HDf5
model = load_model('modelos/model-frutas.h5')

# arquitetura das camadas em JSSON e pesos treinados em HDF5
with open('modelos/architecture-frutas.json') as f:
    model = model_from_json(f.read())
    
model.load_weights('modelos/weights-frutas.h5')

### 4.4 Predição nas imagens de teste

In [ ]:
imagens_teste_path = ["imagens-frutas/validation/Apple Braeburn/7_100.jpg",
                        "imagens-frutas/validation/Avocado/49_100.jpg",
                        "imagens-frutas/validation/Banana/12_100.jpg",
                        "imagens/banana.jpeg"]

lista_imagem = []

for imagem_path in imagens_teste_path:
    imagem = cv2.imread(imagem_path)
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
    imagem = cv2.resize(imagem, (224, 224))
    lista_imagem.append(imagem)

In [ ]:
lista_imagem_array = np.array(lista_imagem, dtype="float")
lista_imagem_array = preprocess_input(lista_imagem_array)

Normalizando as imagens de teste, como neste caso não usamos o gerador do Keras, precisamos ajustar manualmente.

In [ ]:
pred_probs = model.predict(lista_imagem_array)
pred_probs

In [ ]:
plt.figure(figsize=(20,10))

for i, imagem in enumerate(lista_imagem):
    plt.subplot(2,2,i+1)
    plt.imshow(imagem)
    plt.title("{:.0f}% Apple, {:.0f}% Avocado, {:.0f}% Banana".format(100*pred_probs[i,0], 100*pred_probs[i,1], 100*pred_probs[i,2]))

## 5. Classificador de Objetos

É necessário baixar os pesos (modelo de deep-learning) neste link https://pjreddie.com/media/files/yolov3.weights e copiar para  pasta weights. O comando a seguir vai baixar o arquivo de pesos no diretório ```pesos```.

Utilizar esse caminho por ser mais rápido o download.

In [ ]:
!gdown https://drive.google.com/uc?id=1paGkSIW97J-Cc1oLbuYdKkkK4zynq1SK -O pesos/yolov3.weights

Caminho original.

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights -P pesos/

Configurações do modelo.

In [ ]:
# Configurações na rede neural YOLOv3
config_file = 'cfg/yolov3.cfg'
modelo_yolo = Darknet(config_file)

# Pesos pré-treinados
weight_file = 'pesos/yolov3.weights'
modelo_yolo.load_weights(weight_file)

# Rótulos de classes
class_names_file = 'data/coco.names'
class_names = load_class_names(class_names_file)

In [ ]:
# Topologia da rede neural da YOLOv3
modelo_yolo.print_network()

In [ ]:
print("Tamanho da imagem de entrada da rede: " + str(modelo_yolo.width) + " x " + str(modelo_yolo.height) + " pixels.")

In [ ]:
# Carregando imagem para classificar
imagem = cv2.imread("imagens/camara.jpg")

# Convertendo para o espaço de cores RGB
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

# Redimensionando imagem para ser compatível com a primeira camada da rede neural  
imagem_padronizada = cv2.resize(imagem, (modelo_yolo.width, modelo_yolo.height))

plt.figure(figsize=(30,10))

# Exibição das imagens
plt.title("Imagem Original")
plt.imshow(imagem)
plt.show()

plt.title("Imagem Redimensionada")
plt.imshow(imagem_padronizada)
plt.show()

In [ ]:
# Patamar de NMS (Non-Maximum Supression)
# Ajuste de sensibilidade de imagens com baixa luminosidade
nms_thresh = 0.6

# Patamar do IOU (Intersect of Union), indicador se o retângulo 
# de identificação de imagem foi adequadamente desenhado
iou_thresh = 0.4

In [ ]:
# Definindo tamnaho do gráfico
plt.rcParams['figure.figsize'] = [30, 20]

# Deteteção de objetos na imagem
boxes = detect_objects(modelo_yolo, imagem_padronizada, iou_thresh, nms_thresh)

# Objetos encontrados e nível de confiança
print_objects(boxes, class_names)

# Desenho no gráfico com os regângulos e rótulos
plot_boxes(imagem, boxes, class_names, plot_labels = True)

Lista de todos os objetos identificados na imagem.

In [ ]:
list_objects(boxes, class_names)